# Replicate current behaviour for closed shells
Once that works we can move on to checking that open shells work.

* [ ] Use unrestricted method for closed shell

* [ ] Check that values match restricted, incl energy contributions

* [ ] Try with open shell

* [ ] Change driver back to allow choice of restricted scf

* [ ] Insert new functions

* [ ] Update pyscf localizers to do unrestricted


Need to force driver to always use unrestricted method.

In [1]:
from nbed.driver import NbedDriver
from pathlib import Path

args = {
    "geometry": str(Path("molecular_structures/cyclopentane.xyz").absolute()),
    "basis": "STO-3G",
    "xc_functional": "b3lyp",
    "n_active_atoms":2,
    "projector": "mu",
    "localization": "spade",
    "convergence": 1e-6,
    "savefile": None,
    "run_ccsd_emb": True,
    "run_fci_emb": False,
    'charge':0,
    'spin':0,
}

/home/mwilliams/.cache/pypoetry/virtualenvs/nbed-g8dV7HW4-py3.8/lib/python3.8/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'
/home/mwilliams/.cache/pypoetry/virtualenvs/nbed-g8dV7HW4-py3.8/lib/python3.8/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


## Run restricted to get componenets

In [66]:
driver = NbedDriver(**args)

In [67]:
import logging
logger = logging.getLogger(__name__)
from nbed.localizers import Localizer
from typing import Optional, Tuple, Dict
import numpy as np
from pyscf import scf, dft, gto
from scipy import linalg
from nbed.exceptions import NbedConfigError

class OldSPADELocalizer(Localizer):
    """Localizer Class to carry out SPADE"""

    def __init__(
        self,
        pyscf_scf: gto.Mole,
        n_active_atoms: int,
        occ_cutoff: Optional[float] = 0.95,
        virt_cutoff: Optional[float] = 0.95,
        run_virtual_localization: Optional[bool] = False,
    ):
        super().__init__(
            pyscf_scf,
            n_active_atoms,
            occ_cutoff=occ_cutoff,
            virt_cutoff=virt_cutoff,
            run_virtual_localization=run_virtual_localization,
        )

    def _localize(
        self,
    ) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
        """Localise orbitals using SPADE.
        Returns:
            active_MO_inds (np.array): 1D array of active occupied MO indices
            enviro_MO_inds (np.array): 1D array of environment occupied MO indices
            c_active (np.array): C matrix of localized occupied active MOs (columns define MOs)
            c_enviro (np.array): C matrix of localized occupied ennironment MOs
            c_loc_occ (np.array): full C matrix of localized occupied MOs
        """
        logger.info("Localising with SPADE.")
        n_occupied_orbitals = np.count_nonzero(self._global_ks.mo_occ)

        occupied_orbitals = self._global_ks.mo_coeff[:, :n_occupied_orbitals]

        n_act_aos = self._global_ks.mol.aoslice_by_atom()[self._n_active_atoms - 1][-1]
        logger.debug(f"{n_act_aos} active AOs.")

        ao_overlap = self._global_ks.get_ovlp()

        # Orbital rotation and partition into subsystems A and B
        # rotation_matrix, sigma = embed.orbital_rotation(occupied_orbitals,
        #    n_act_aos, ao_overlap)

        rotated_orbitals = (
            linalg.fractional_matrix_power(ao_overlap, 0.5) @ occupied_orbitals
        )
        _, sigma, right_vectors = linalg.svd(rotated_orbitals[:n_act_aos, :])

        logger.debug(f"Singular Values: {sigma}")

        # n_act_mos, n_env_mos = embed.orbital_partition(sigma)
        # Prevents an error with argmax
        if len(sigma) == 1:
            n_act_mos = 1
        else:
            value_diffs = sigma[:-1] - sigma[1:]
            n_act_mos = np.argmax(value_diffs) + 1
        n_env_mos = n_occupied_orbitals - n_act_mos
        logger.debug(f"{n_act_mos} active MOs.")
        logger.debug(f"{n_env_mos} environment MOs.")

        # get active and enviro indices
        active_MO_inds = np.arange(n_act_mos)
        enviro_MO_inds = np.arange(n_act_mos, n_act_mos + n_env_mos)

        # Defining active and environment orbitals and density
        c_active = occupied_orbitals @ right_vectors.T[:, :n_act_mos]
        c_enviro = occupied_orbitals @ right_vectors.T[:, n_act_mos:]
        c_loc_occ = occupied_orbitals @ right_vectors.T

        # storing condition used to select env system
        self.enviro_selection_condition = sigma

        alpha = active_MO_inds, enviro_MO_inds, c_active, c_enviro, c_loc_occ
        beta = None

        return (alpha, beta)

    def run(self, sanity_check: bool = False) -> None:
        """Function that runs localization.

        Args:
            sanity_check (bool): optional flag to check denisty matrices and electron number after orbital localization
                                 makes sense
        """
        alpha, beta = self._localize()

        (
            self.active_MO_inds,
            self.enviro_MO_inds,
            self.c_active,
            self.c_enviro,
            self._c_loc_occ,
        ) = alpha

        self.dm_active = 2.0 * self.c_active @ self.c_active.T
        self.dm_enviro = 2.0 * self.c_enviro @ self.c_enviro.T

        self.beta_active_MO_inds = None
        self.beta_enviro_MO_inds = None
        self.beta_c_active = None
        self.beta_c_enviro = None
        self._beta_c_loc_occ = None
        self.beta_dm_active = None
        self.beta_dm_enviro = None

        if beta is not None:
            # Weight the DMs by 1 for unrestricted to combine them.
            self.dm_active *= 0.5
            self.dm_enviro *= 0.5

            (
                self.beta_active_MO_inds,
                self.beta_enviro_MO_inds,
                self.beta_c_active,
                self.beta_c_enviro,
                self._beta_c_loc_occ,
            ) = beta

            self.beta_dm_active = self.beta_c_active @ self.beta_c_active.T
            self.beta_dm_enviro = self.beta_c_enviro @ self.beta_c_enviro.T

        if sanity_check is True:
            self._check_values()

        if self._run_virtual_localization is True:
            logger.error("Virtual localization is not implemented.")
            # c_virtual = self._localize_virtual_orbs()
            # logger.error("Defualting to unlocalized virtual orbitals.")
            # c_virtual = self._global_ks.mo_coeff[:, self._global_ks.mo_occ < 2]
        else:
            logger.debug("Not localizing virtual orbitals.")
            # appends standard virtual orbitals from SCF calculation (NOT localized in any way)
            # c_virtual = self._global_ks.mo_coeff[:, self._global_ks.mo_occ < 2]

        # Unused
        # self.c_loc_occ_and_virt = np.hstack((self._c_loc_occ, c_virtual))

        logger.debug("Localization complete.")

In [69]:
import numpy as np
driver = NbedDriver(**args)

## Combine the C matrices and occupations into one to use restricted localizer
if driver._global_ks.mo_coeff.shape[0] == 2:
    driver._global_ks.mo_coeff = np.sum(driver._global_ks.mo_coeff, axis=0)/2
    driver._global_ks.mo_occ = np.sum(driver._global_ks.mo_occ, axis=0)
    
print(driver._global_ks.mo_coeff.shape)
old = OldSPADELocalizer(driver._global_ks, driver.n_active_atoms)
driver.localized_system = old
print(old.active_MO_inds)
print(old.enviro_MO_inds)

(35, 35)
[0 1 2 3 4]
[ 5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


In [70]:
from nbed.localizers import SPADELocalizer


class UnrestrictedDriver(NbedDriver):
    def __init__(
        self,
        geometry: str,
        n_active_atoms: int,
        basis: str,
        xc_functional: str,
        projector: str,
        localization: Optional[str] = "spade",
        convergence: Optional[float] = 1e-6,
        charge: Optional[int] = 0,
        spin: Optional[int] = 0,
        mu_level_shift: Optional[float] = 1e6,
        run_ccsd_emb: Optional[bool] = False,
        run_fci_emb: Optional[bool] = False,
        run_virtual_localization: Optional[bool] = False,
        run_dft_in_dft: Optional[bool] = False,
        max_ram_memory: Optional[int] = 4000,
        pyscf_print_level: int = 1,
        savefile: Optional[Path] = None,
        unit: Optional[str] = "angstrom",
        occupied_threshold: Optional[float] = 0.95,
        virtual_threshold: Optional[float] = 0.95,
        init_huzinaga_rhf_with_mu: bool = False,
        max_hf_cycles: int = 50,
        max_dft_cycles: int = 50,
    ):
        """Initialise class."""
        logger.debug("Initialising driver.")
        config_valid = True
        if projector not in ["mu", "huzinaga", "both"]:
            logger.error(
                "Invalid projector %s selected. Choose from 'mu' or 'huzinzaga'.",
                projector,
            )
            config_valid = False

        if localization not in ["spade", "ibo", "boys", "pipek-mezey"]:
            logger.error(
                "Invalid localization method %s. Choose from 'ibo','boys','pipek-mezey' or 'spade'.",
                localization,
            )
            config_valid = False

        if not config_valid:
            logger.error("Invalid config.")
            raise NbedConfigError("Invalid config.")

        self.geometry = geometry
        self.n_active_atoms = n_active_atoms
        self.basis = basis.lower()
        self.xc_functional = xc_functional.lower()
        self.projector = projector.lower()
        self.localization = localization.lower()
        self.convergence = convergence
        self.charge = charge
        self.spin = spin
        self.mu_level_shift = mu_level_shift
        self.run_ccsd_emb = run_ccsd_emb
        self.run_fci_emb = run_fci_emb
        self.run_virtual_localization = run_virtual_localization
        self.run_dft_in_dft = run_dft_in_dft
        self.max_ram_memory = max_ram_memory
        self.pyscf_print_level = pyscf_print_level
        self.savefile = savefile
        self.unit = unit
        self.occupied_threshold = occupied_threshold
        self.virtual_threshold = virtual_threshold
        self.max_hf_cycles = max_hf_cycles
        self.max_dft_cycles = max_dft_cycles

        self._check_active_atoms()
        self.localized_system = None
        self.two_e_cross = None
        self._dft_potential = None

        self._restricted_scf = False

        # self.embed(init_huzinaga_rhf_with_mu=init_huzinaga_rhf_with_mu) # TODO uncomment.
        logger.debug("Driver initialisation complete.")


## If the C matrix is combined before localisation, we get the right DFT components

In [75]:
driver = UnrestrictedDriver(**args)

## Combine the C matrices and occupations into one to use restricted localizer
if driver._restricted_scf is False:
    driver._global_ks.mo_coeff = np.sum(driver._global_ks.mo_coeff, axis=0)/2
    driver._global_ks.mo_occ = np.sum(driver._global_ks.mo_occ, axis=0)
    
print(driver._global_ks.mo_coeff.shape)
old = OldSPADELocalizer(driver._global_ks, driver.n_active_atoms)
driver.localized_system = old
print(old.active_MO_inds)
print(old.enviro_MO_inds)

(35, 35)
(35, 35)
[0 1 2 3 4]
[ 5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


## If the density matrices are combined after localisation, it also passes!

In [78]:
driver = UnrestrictedDriver(**args)

## Combine the C matrices and occupations into one to use restricted localizer
print(driver._global_ks.mo_coeff.shape)
old = SPADELocalizer(driver._global_ks, driver.n_active_atoms)
driver.localized_system = old
print(old.active_MO_inds)
print(old.enviro_MO_inds)

driver.localized_system.dm_active += driver.localized_system.beta_dm_active
driver.localized_system.dm_enviro += driver.localized_system.beta_dm_enviro

(2, 35, 35)
occupancy=array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.])
occupancy=array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.])
[0 1 2 3 4]
[ 5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


In [79]:
"""Function to perform subsystem RKS DFT calculation."""
logger.debug("Calculating active and environment subsystem terms.")

def _ks_components(
    ks_system: Localizer,
    subsystem_dm: np.ndarray,
) -> Tuple[float, float, np.ndarray, np.ndarray, np.ndarray]:
    """Calculate the components of subsystem energy from a RKS DFT calculation.

    For a given density matrix this function returns the electronic energy, exchange correlation energy and
    J,K, V_xc matrices.

    Args:
        dm_matrix (np.ndarray): density matrix (to calculate all matrices from)

    Returns:
        Energy_elec (float): DFT energy defubed by input density matrix
        e_xc (float): exchange correlation energy defined by input density matrix
        J_mat (np.ndarray): J_matrix defined by input density matrix
    """
    logger.debug("Finding subsystem RKS componenets.")
    # It seems that PySCF lumps J and K in the J array
    two_e_term = ks_system.get_veff(dm=subsystem_dm)
    j_mat = two_e_term.vj
    # k_mat = np.zeros_like(j_mat)

    e_xc = two_e_term.exc
    # v_xc = two_e_term - j_mat

    energy_elec = (
        np.einsum("ij,ji->", ks_system.get_hcore(), subsystem_dm)
        + two_e_term.ecoul
        + two_e_term.exc
    )

    # if check_E_with_pyscf:
    #     energy_elec_pyscf = driver._global_ks.energy_elec(dm=dm_matrix)[0]
    #     if not np.isclose(energy_elec_pyscf, energy_elec):
    #         raise ValueError("Energy calculation incorrect")
    logger.debug(f"Subsystem RKS components found.")
    return energy_elec, e_xc, j_mat

(driver.e_act, e_xc_act, j_act) = _ks_components(
    driver._global_ks, driver.localized_system.dm_active
)
(driver.e_env, e_xc_env, j_env) = _ks_components(
    driver._global_ks, driver.localized_system.dm_enviro
)
# print(f"{driver.e}")
# Computing cross subsystem terms
logger.debug("Calculating two electron cross subsystem energy.")

two_e_term_total = driver._global_ks.get_veff(
    dm=driver.localized_system.dm_active + driver.localized_system.dm_enviro
)
e_xc_total = two_e_term_total.exc

j_cross = 0.5 * (
    np.einsum("ij,ij", driver.localized_system.dm_active, j_env)
    + np.einsum("ij,ij", driver.localized_system.dm_enviro, j_act)
)
# Because of projection
k_cross = 0.0

xc_cross = e_xc_total - e_xc_act - e_xc_env
print(f"{xc_cross=}")

# overall two_electron cross energy
driver.two_e_cross = j_cross + k_cross + xc_cross

energy_DFT_components = (
    driver.e_act + driver.e_env + driver.two_e_cross + driver._global_ks.energy_nuc()
)
print("RKS components")
print(f"{driver.e_act=},{driver.e_env=},{driver.two_e_cross=},{driver._global_ks.energy_nuc()=}")
print(driver._global_ks.scf_summary)
if not np.isclose(energy_DFT_components, driver._global_ks.e_tot):
    print(f"{energy_DFT_components=}")
    print(f"{driver._global_ks.e_tot=}")
    logger.error(
        "DFT energy of localized components not matching supersystem DFT."
    )
    raise ValueError(
        "DFT energy of localized components not matching supersystem DFT."
    )


xc_cross=-0.34977914311294356
RKS components
driver.e_act=-129.12067920087077,driver.e_env=-329.7853053598932,driver.two_e_cross=75.71842178308916,driver._global_ks.energy_nuc()=189.1078142252856
{'e1': -635.5196655298083, 'coul': 283.13322494869374, 'exc': -30.80112219707005, 'nuc': 189.1078142252856}


# Localize separately

Create a driver to force unrestricted even for closed shells.

In [52]:

driver = NbedDriver(**args)
new = SPADELocalizer(driver._global_ks, driver.n_active_atoms)
print(new.active_MO_inds)
print(new.beta_active_MO_inds)
print(new.enviro_MO_inds)
print(new.beta_enviro_MO_inds)

occupancy=array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.])
[0 1 2 3 4]
None
[ 5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
None


# Mock Driver.embed()

In [53]:
import logging
logger = logging.getLogger(__name__)
from nbed.localizers import Localizer
from typing import Optional, Tuple, Dict
import numpy as np
from pyscf import scf, dft, gto

init_huzinaga_rhf_with_mu = False

"""Generate embedded Hamiltonian.

Note run_mu_shift (bool) and run_huzinaga (bool) flags define which method to use (can be both)
This is done when object is initialized.
"""
logger.debug("Embedding molecule.")
localized_system = driver._localize()
driver.localized_system = localized_system

# logger.info(localized_system.active_MO_inds, localized_system.beta_active_MO_inds)
# logger.info(localized_system.enviro_MO_inds, localized_system.beta_enviro_MO_inds)

e_nuc = driver._global_ks.energy_nuc()

# Run subsystem DFT (calls localized rks)
#driver._subsystem_dft()


occupancy=array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.])


In [60]:
assert driver._restricted_scf
assert localized_system._restricted_scf
localized_system._global_ks.scf_summary

{'e1': -635.5197347973623,
 'coul': 283.133296927738,
 'exc': -30.801124900358385,
 'nuc': 189.1078142252856}

In [80]:
"""Function to perform subsystem RKS DFT calculation."""
logger.debug("Calculating active and environment subsystem terms.")

def _ks_components(
    ks_system: Localizer,
    subsystem_dm: np.ndarray,
) -> Tuple[float, float, np.ndarray, np.ndarray, np.ndarray]:
    """Calculate the components of subsystem energy from a RKS DFT calculation.

    For a given density matrix this function returns the electronic energy, exchange correlation energy and
    J,K, V_xc matrices.

    Args:
        dm_matrix (np.ndarray): density matrix (to calculate all matrices from)

    Returns:
        Energy_elec (float): DFT energy defubed by input density matrix
        e_xc (float): exchange correlation energy defined by input density matrix
        J_mat (np.ndarray): J_matrix defined by input density matrix
    """
    logger.debug("Finding subsystem RKS componenets.")
    # It seems that PySCF lumps J and K in the J array
    # need to access the potential for the right subsystem for unrestricted
    two_e_term = ks_system.get_veff(dm=subsystem_dm)
    j_mat = two_e_term.vj
    # k_mat = np.zeros_like(j_mat)

    e_xc = two_e_term.exc
    # v_xc = two_e_term - j_mat

    h1e = (
        np.einsum("ij,ji->", ks_system.get_hcore(), subsystem_dm)
        #+ two_e_term.ecoul
        #+ two_e_term.exc
    )

    # if check_E_with_pyscf:
    #     energy_elec_pyscf = driver._global_ks.energy_elec(dm=dm_matrix)[0]
    #     if not np.isclose(energy_elec_pyscf, energy_elec):
    #         raise ValueError("Energy calculation incorrect")
    logger.debug(f"Subsystem RKS components found.")
    return h1e, e_xc, j_mat, two_e_term.ecoul

(alpha_e_act, alpha_e_xc_act, alpha_j_act, alpha_ecoul_act) = _ks_components(
    driver._global_ks, driver.localized_system.dm_active
)
print(alpha_e_act, alpha_e_xc_act, alpha_ecoul_act)
(alpha_e_env, alpha_e_xc_env, alpha_j_env, alpha_ecoul_env) = _ks_components(
    driver._global_ks, driver.localized_system.dm_enviro
)
print(alpha_e_env, alpha_e_xc_env, alpha_ecoul_env)
driver.e_act = alpha_e_act
driver.e_env = alpha_e_env

if not driver._restricted_scf:
    (beta_e_act, beta_e_xc_act, beta_j_act, beta_ecoul_act) = _ks_components(
        driver._global_ks, driver.localized_system.beta_dm_active
    )
    print(beta_e_act, beta_e_xc_act, beta_ecoul_act)
    (beta_e_env, beta_e_xc_env, beta_j_env, beta_ecoul_env) = _ks_components(
        driver._global_ks, driver.localized_system.beta_dm_enviro
    )
    print(beta_e_env, beta_e_xc_env, beta_ecoul_env)
    driver.e_act += beta_e_act
    driver.e_env += beta_e_env

# Computing cross subsystem terms
logger.debug("Calculating two electron cross subsystem energy.")
total_dm = driver.localized_system.dm_active + driver.localized_system.dm_enviro

if not driver._restricted_scf:
    total_dm += driver.localized_system.beta_dm_active + driver.localized_system.beta_dm_enviro
    
two_e_term_total = driver._global_ks.get_veff(
    dm=total_dm)
e_xc_total = two_e_term_total.exc

# if not driver._restricted_scf:
#     beta_two_e_term_total = driver._global_ks.get_veff(
#         dm=driver.localized_system.beta_dm_active + driver.localized_system.beta_dm_enviro
#     )
#     e_xc_total = (e_xc_total + beta_two_e_term_total.exc)/2

j_cross = 0.5 * (
    np.einsum("ij,ij", driver.localized_system.dm_active, alpha_j_env)
    + np.einsum("ij,ij", driver.localized_system.dm_enviro, alpha_j_act)
)
if not driver._restricted_scf:
    beta_j_cross = 0.5 * (
        np.einsum("ij,ij", driver.localized_system.dm_active, beta_j_env)
        + np.einsum("ij,ij", driver.localized_system.dm_enviro, beta_j_act)
        + np.einsum("ij,ij", driver.localized_system.beta_dm_active, beta_j_env)
        + np.einsum("ij,ij", driver.localized_system.beta_dm_enviro, beta_j_act)
        + np.einsum("ij,ij", driver.localized_system.beta_dm_active, alpha_j_env)  
        + np.einsum("ij,ij", driver.localized_system.beta_dm_enviro, alpha_j_act)
    )
    j_cross += beta_j_cross

# Because of projection
k_cross = 0.0

xc_cross = e_xc_total - alpha_e_xc_act - alpha_e_xc_env

if not driver._restricted_scf:
    xc_cross = xc_cross -beta_e_xc_act - beta_e_xc_env

print(f'{xc_cross=}, {e_xc_total=}, {alpha_e_xc_act=}, {alpha_e_xc_env=}, {beta_e_xc_act=}, {beta_e_xc_env=}')
# if not driver._restricted_scf:
    # xc_cross = beta_e_xc_act - beta_e_xc_env
# driver.e_act=-129.12067009902808      driver.e_env=-329.78536772993164
# e_xc_total=-30.801124900358385, e_xc_act=-6.870585748333742, e_xc_env=-23.580761027571594
# j_cross=76.06825318343016 k_cross=0.0 xc_cross=-0.34977812445304934

# driver.e_act=-143.82987154686245 driver.e_env=-411.35820634557757
# e_xc_total=-30.80112219655959, alpha_e_xc_act=-2.6149703376981797, alpha_e_xc_env=-8.98550003899027, 
# j_cross=76.06820092620214, k_cross=0.0, xc_cross=-19.20065181987114

# overall two_electron cross energy
driver.two_e_cross = j_cross + k_cross + xc_cross
print(f"{driver.e_act=} {driver.e_env=}")
print(f"{e_xc_total=}, {alpha_e_xc_act=}, {alpha_e_xc_env=}, {j_cross=}, {k_cross=}, {xc_cross=}")

energy_DFT_components = (
    driver.e_act + driver.e_env + driver.two_e_cross + driver._global_ks.energy_nuc()
)
logger.info("RKS components")
logger.info(f'{driver.e_act=} {driver.e_env=} {driver.two_e_cross=}')
print(f'{driver._global_ks.scf_summary}')
print(f'{energy_DFT_components=}')
print(driver._global_ks.e_tot)
if not np.isclose(energy_DFT_components, driver._global_ks.e_tot):
    logger.error(
        "DFT energy of localized components not matching supersystem DFT."
    )
    raise ValueError(
        "DFT energy of localized components not matching supersystem DFT."
    )


-129.12067920087077 -6.870575593286572 32.69966179970581
-329.78530535989324 -23.58076746016005 174.36536222278576
-71.91488938928742 -2.6149667018569187 8.17489635009535
-205.67895246974126 -8.985487296065536 43.591235065814914


1970-01-01 05:54:44,691: __main__: ERROR: DFT energy of localized components not matching supersystem DFT.


xc_cross=-13.17656269570363, e_xc_total=-55.22835974707271, alpha_e_xc_act=-6.870575593286572, alpha_e_xc_env=-23.58076746016005, beta_e_xc_act=-2.6149667018569187, beta_e_xc_env=-8.985487296065536
driver.e_act=-201.0355685901582 driver.e_env=-535.4642578296346
e_xc_total=-55.22835974707271, alpha_e_xc_act=-6.870575593286572, alpha_e_xc_env=-23.58076746016005, j_cross=171.1533374121301, k_cross=0.0, xc_cross=-13.17656269570363
{'e1': -635.5196655298083, 'coul': 283.13322494869374, 'exc': -30.80112219707005, 'nuc': 189.1078142252856}
energy_DFT_components=-389.41523747808066
-194.07974855289905


ValueError: DFT energy of localized components not matching supersystem DFT.

xc_cross=-0.34977812445304934
RKS components
driver.e_act=-129.1206700990281,driver.e_env=-329.7853677299315,driver.two_e_cross=75.71847505897706,driver._global_ks.energy_nuc()=189.1078142252856
{'e1': -635.5197347973617, 'coul': 283.1332969277376, 'exc': -30.801124900358353, 'nuc': 189.1078142252856}

In [ ]:
"""Function to perform subsystem RKS DFT calculation."""
logger.debug("Calculating active and environment subsystem terms.")

def _rks_components(
    rks_system: Localizer, subsystem_dm: np.ndarray,
) -> Tuple[float, float, np.ndarray, np.ndarray, np.ndarray]:
    """Calculate the components of subsystem energy from a RKS DFT calculation.
    For a given density matrix this function returns the electronic energy, exchange correlation energy and
    J,K, V_xc matrices.
    Args:
        dm_matrix (np.ndarray): density matrix (to calculate all matrices from)
    Returns:
        Energy_elec (float): DFT energy defubed by input density matrix
        e_xc (float): exchange correlation energy defined by input density matrix
        J_mat (np.ndarray): J_matrix defined by input density matrix
    """
    dm_matrix = subsystem_dm
    # It seems that PySCF lumps J and K in the J array
    two_e_term = rks_system.get_veff(dm=dm_matrix)
    j_mat = two_e_term.vj
    # k_mat = np.zeros_like(j_mat)

    e_xc = two_e_term.exc
    # v_xc = two_e_term - j_mat

    energy_elec = (
        np.einsum("ij,ji->", rks_system.get_hcore(), dm_matrix)
        + two_e_term.ecoul
        + two_e_term.exc
    )

    # if check_E_with_pyscf:
    #     energy_elec_pyscf = driver._global_ks.energy_elec(dm=dm_matrix)[0]
    #     if not np.isclose(energy_elec_pyscf, energy_elec):
    #         raise ValueError("Energy calculation incorrect")

    return energy_elec, e_xc, j_mat

(driver.e_act, e_xc_act, j_act) = _rks_components(
    driver._global_ks, driver.localized_system.dm_active
)
(driver.e_env, e_xc_env, j_env) = _rks_components(
    driver._global_ks, driver.localized_system.dm_enviro
)
# Computing cross subsystem terms
logger.debug("Calculating two electron cross subsystem energy.")

two_e_term_total = driver._global_ks.get_veff(
    dm=driver.localized_system.dm_active + driver.localized_system.dm_enviro
)
e_xc_total = two_e_term_total.exc

j_cross = 0.5 * (
    np.einsum("ij,ij", driver.localized_system.dm_active, j_env)
    + np.einsum("ij,ij", driver.localized_system.dm_enviro, j_act)
)
# Because of projection
k_cross = 0.0

xc_cross = e_xc_total - e_xc_act - e_xc_env

# overall two_electron cross energy
driver.two_e_cross = j_cross + k_cross + xc_cross
print(f"{e_xc_total=}, {alpha_e_xc_act=}, {alpha_e_xc_env=}, {j_cross=}, {k_cross=}, {xc_cross=}")

energy_DFT_components = (
    driver.e_act + driver.e_env + driver.two_e_cross + driver._global_ks.energy_nuc()
)
logger.info("RKS components")
logger.info(driver.e_act)
logger.info(driver.e_env)
logger.info(driver.two_e_cross)
logger.info(driver._global_ks.energy_nuc())
print(energy_DFT_components, driver._global_ks.e_tot)
if not np.isclose(energy_DFT_components, driver._global_ks.e_tot):

    raise ValueError(
        "DFT energy of localized components not matching supersystem DFT"
    )

In [ ]:

logger.debug("Getting global DFT potential to optimize embedded calc in.")
g_act_and_env = driver._global_ks.get_veff(
    dm=(localized_system.dm_active + localized_system.dm_enviro)
)
g_act = driver._global_ks.get_veff(dm=localized_system.dm_active)
driver._dft_potential = g_act_and_env - g_act
logger.info(f"DFT potential average {np.mean(driver._dft_potential)}.")

# To add a projector, put it in this dict with a function
# if we want any more it's also time to turn them into a class
embeddings: Dict[str, callable] = {
    "huzinaga": driver._huzinaga_embed,
    "mu": driver._mu_embed,
}
if driver.projector in embeddings:
    embeddings = {driver.projector: embeddings[driver.projector]}

# This is reverse so that huz can be initialised with mu
for name in sorted(embeddings, reverse=True):
    logger.debug(f"Runnning embedding with {name} projector.")
    setattr(driver, "_" + name, {})
    result = getattr(driver, "_" + name)

    embedding_method = embeddings[name]
    local_rhf = driver._init_local_rhf()

    if init_huzinaga_rhf_with_mu and (name == "huzinaga"):
        logger.debug("Initializing huzinaga with mu-shift.")
        # seed huzinaga calc with mu result!
        result["v_emb"], result["scf"] = embedding_method(
            local_rhf, dmat_initial_guess=driver._mu["scf"].make_rdm1()
        )
    else:
        result["v_emb"], result["scf"] = embedding_method(local_rhf)

    result["mo_energies_emb_pre_del"] = local_rhf.mo_energy
    result["scf"] = driver._delete_environment(result["scf"], name)
    result["mo_energies_emb_post_del"] = local_rhf.mo_energy

    logger.info(f"V emb mean {name}: {np.mean(result['v_emb'])}")

    # calculate correction
    result["correction"] = np.einsum(
        "ij,ij", result["v_emb"], localized_system.dm_active
    )
    result["e_rhf"] = (
        result["scf"].e_tot
        + driver.e_env
        + driver.two_e_cross
        - result["correction"]
    )
    logger.info(f"RHF energy: {result['e_rhf']}")

    # classical energy
    result["classical_energy"] = (
        driver.e_env + driver.two_e_cross + e_nuc - result["correction"]
    )

    # Calculate ccsd or fci energy
    if driver.run_ccsd_emb is True:
        logger.debug("Performing CCSD-in-DFT embedding.")
        ccsd_emb, e_ccsd_corr = driver._run_emb_CCSD(
            result["scf"], frozen_orb_list=None
        )
        result["e_ccsd"] = (
            ccsd_emb.e_hf
            + e_ccsd_corr
            + driver.e_env
            + driver.two_e_cross
            - result["correction"]
        )
        logger.info(f"CCSD Energy {name}:\t{result['e_ccsd']}")

    if driver.run_fci_emb is True:
        logger.debug("Performing FCI-in-DFT embedding.")
        fci_emb = driver._run_emb_FCI(result["scf"], frozen_orb_list=None)
        result["e_fci"] = (
            (fci_emb.e_tot)
            + driver.e_env
            + driver.two_e_cross
            - result["correction"]
        )
        logger.info(f"FCI Energy {name}:\t{result['e_fci']}")

    if driver.run_dft_in_dft is True:
        did = driver.embed_dft_in_dft(driver._global_ks.xc, embedding_method)
        result["e_dft_in_dft"] = did["e_rks"]

if driver.projector == "both":
    logger.warning(
        "Outputting both mu and huzinaga embedding results as tuple."
    )
    driver.embedded_scf = (
        driver._mu["scf"],
        driver._huzinaga["scf"],
    )
    driver.classical_energy = (
        driver._mu["classical_energy"],
        driver._huzinaga["classical_energy"],
    )
elif driver.projector == "mu":
    driver.embedded_scf = driver._mu["scf"]
    driver.classical_energy = driver._mu["classical_energy"]
elif driver.projector == "huzinaga":
    driver.embedded_scf = driver._huzinaga["scf"]
    driver.classical_energy = driver._huzinaga["classical_energy"]

logger.info("Embedding complete.")